<a href="https://colab.research.google.com/github/18520395/CS114.K21/blob/master/sarcasm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
#import libraries
import numpy as np
import pandas as pd
import sklearn as sk
import matplotlib as plt

In [ ]:
#load libraries
from pandas import read_csv
from sklearn.model_selection import train_test_split

In [ ]:
#load dataset
